# Principal Component Analysis

### Objective

This notebook aims to use **principal component analysis (PCA)** on decathlon's data, which refers to men's  performance during two athletic meetings: [2004 Summer Olympics]
(https://en.wikipedia.org/wiki/Athletics_at_the_2004_Summer_Olympics_%E2%80%93_Men%27s_decathlon) and 2004 [Decastar](https://en.wikipedia.org/wiki/Décastar).

This notebook was inspired by Chloé-Agathe Azencott and FactoMineR.

### Requirements

1. This notebook was created using
  * python 3.7.1
  * numpy 1.15.4
  * matplotlib 3.0.2
  * scikit-learn 0.20.1
  * seaborn 0.9.0

You can check your Python version by running
```python
import sys
print(sys.version)
```

and the version of any module by running
```python
import <module name>
print(<module name>.__version__)
```

### Please complete all the \# TODOs

## Loading the libraries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

We use `matplotlib` for data visualization together with its built-in interface called `pyplot`. Please check https://matplotlib.org/contents.html for more information.


## Decathlon's dataset

The data are available at http://factominer.free.fr/factomethods/datasets/decathlon.txt

### Data description

* The dataset consists of 41 rows and 13 columns.
* The first row is a header describing the content of the columns and the remaining rows refer to the 40 observations (athletes) considered in this dataset.
* Columns 1 to 12 are continuous variables: the first ten columns correspond to the performance of the athletes for each event of the decathlon and columns 11 and 12 correspond respectively to the rank and the points obtained.
* The last column is a categorical variable corresponding to the athletic meeting (2004 Olympic Games or 2004 Decastar).


### Loading the data





In [ ]:
decathlon_df = pd.read_csv('http://factominer.free.fr/factomethods/datasets/decathlon.txt', sep = '\t')

### Checking the shape of the data

In [ ]:
decathlon_df.shape

### Checking the name of the columns

In [ ]:
decathlon_df.columns

### Visualizing the first five records

In [ ]:
decathlon_df.head()  

### Accessing data

* We can select a column by name. Note the returned object is also a pandas object (a *series*--a single-columned DataFrame), so we can use the `head()` function to view the first few rows only.

In [ ]:
decathlon_df['100m'].head()

* Or a list for multiple columns.

In [ ]:
decathlon_df[['100m', '400m']].head()

* We can check the unique values of a feature (i.e., column)

In [ ]:
decathlon_df['Competition'].unique()

* We can select rows satisfying a given condition(s) by passing a boolean series.

In [ ]:
decathlon_df[my_data['Competition']=='OlympicG'].head()

* To *index* a row, we can use the data frame's `loc` object. This behaves like a dictionary whose keys are the data frame's index.

In [ ]:
decathlon_df.loc['CLAY']

* We can get an overview of the data

In [ ]:
decathlon_df.describe()  # describe the data

### Manipulating data

In [ ]:
X = decathlon_df.values  # get the content of the dataframe as a numpy array

In [ ]:
decathlon_df.dtypes  # get the type (dtype) of each column

### Visualisation

In [ ]:
# basic visualization: athletes' performances depending on two disciplines
decathlon_df.plot(kind='scatter', x='400m', y='Shot.put', s=50, color='blue')

In [ ]:
# A scatterplot matrix allows us to visualize:
#   * on the diagonal, the density estimation for each of the features
#   * on each of the off-diagonal plots, a scatterplot between two of the features. 
#     Each dot represents a sample.

from pandas.plotting import scatter_matrix
scatter_matrix(decathlon_df.get(['Shot.put','High.jump', '400m']), alpha=0.2,
               figsize=(6, 6), diagonal='kde');

In [ ]:
# fancy plot with seaborn : https://seaborn.pydata.org/
import seaborn.apionly as sns
sns.set_style('whitegrid')

sns.jointplot('Shot.put', 'High.jump', data = decathlon_df, 
              kind='kde', height=6, space=0, color='b')

# loooking at correlated features
sns.jointplot('Shot.put', 'Discus', data = decathlon_df, 
              kind='reg', height=6, space=0, color='b')

### Cleaning data

In [ ]:
# Remove columns we don't need to represent the athletes
new_decathlon_df = decathlon_df.drop(['Points', 'Rank', 'Competition'], axis=1)

# Verify new column headers
new_decathlon_df.columns

### Find the first two principal components of the data

We will first implement PCA ourselves. Recall PCA of $\mathbf{X} \in \mathbb{R}^{N \times D}$ finds $M$ principal components as the coluns of $\mathbf{W} \in \mathbb{R}^{D \times M}$ and a matrix of projected data $\mathbf{Z} \in \mathbb{R}^{N \times M}$, such that $\mathbf{Z}\mathbf{W}^T$ minimises the error as a reconstruction of $\mathbf{X}$ for the choice of $M$ (equivalently, the factorisation maximally explains the variance in $\mathbf{X}$). The principal components $\mathbf{W}$, correspond to the $M$ largest eigenvectors of the empirical covariance matrix $\boldsymbol\Sigma = \frac{1}{N}\mathbf{X}^T\mathbf{X}$.

**Question:** Recall that PCA works best on standardised data (mean 0, standard deviation 1). Standardise the data. 

**Hint**: you should aim to used numpy's vector-based operations.

In [ ]:
# transform the new dataframe to in a numpy array
X = new_decathlon_df.values

# TODO: standardise the data

**Question:** Find the two first components and project the data. These are the two largest eigenvectors of the empirical covariance matrix, $\boldsymbol\Sigma = \frac{1}{N}\mathbf{X}^T\mathbf{X}$.

In [ ]:
from numpy import linalg

# TODO: calculate the the covariance matrix with numpy

# TODO: find its two first principal components

# TODO: project X onto the principal components


Let's display the projected data. We use a jupyter magic command to display plots inline automatically.

In [ ]:
# create figure and axis objects
fig, ax = plt.subplots(figsize=(6, 5))
# create scatterplot on axis N.B. we record the return value to feed to the colorbar
cax = ax.scatter(X_projected[:, 0], X_projected[:, 1], c = decathlon_df['Rank'],
                 cmap=plt.get_cmap('viridis'))
# Set axis limits
ax.set_xlim([-5.5, 5.5])
ax.set_ylim([-4, 4])
# Create color bar
plt.colorbar(cax, label='Rank')

**Question:** can you recognize a pattern ? Describe what you see.

__Answer__:

### Use scikit-learn to find the PCs

Here, we use scikit-learn to compute the principal components, and compare the results to what we have gotten so far. A useful resource is the online documentation: http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
from sklearn import decomposition, preprocessing

std_scale = preprocessing.StandardScaler().fit(X)
X_scaled = std_scale.transform(X)

pca = decomposition.PCA(n_components=2)
pca.fit(X_scaled)

**Question:** Use `pca.transform` to project the data onto its principal components.

In [ ]:
# TODO: project X on principal components

`pca.explained_variance_ratio_` gives the percentage of variance explained by each of the components.

In [ ]:
print(pca.explained_variance_ratio_)

**Question:** How is `pca.explained_variance_ratio_` computed? Check this is the case by computing it yourself.

In [ ]:
tot_var = np.var(X_scaled, axis=0).sum()
print ( (1 / tot_var) * np.var(X_projected, axis=0) )

Project the data onto its principal components after standardizing.
 
**Question:** Plot the fraction of variance explained by each of the first 10 principal components.

In [ ]:
# TODO: calculate the first ten components

In [ ]:
plt.bar(np.arange(10), pca.explained_variance_ratio_, color='blue')
plt.xlim([-1, 9])
plt.xlabel("Number of PCs", fontsize=16)
plt.ylabel("Fraction of variance explained", fontsize=16)

To better understand the information captured by the principal components, we can consider  `pca.components_`. These are the columns of $\mathbf{W}$ (for $M = 2$).

In [ ]:
pcs = pca.components_
print pcs[0]

We can display each row of $\mathbf{W}$ in a 2D plot whose x-axis gives its contribution to the first component and y-axis to the second component. Note, whereas before we were visualising the projected data, $\mathbf{Z}$, now we are visualising the projections, $\mathbf{W}$. This indicates how the features cluster i.e. if a pair of feature projections are close, observations will tend to be similarly-valued over those features.

In [ ]:
fig = plt.figure(figsize=(6, 5))
ax = fig.add_subplot(1, 1, 1)
ax.set_xlim([-0.7, 0.7])
ax.set_ylim([-0.7, 0.7])

for i, (x, y) in enumerate(zip(pcs[0, :], pcs[1, :])):
    # plot line between origin and point (x, y)
    ax.plot([0, x], [0, y], color='k')
    # display the label of the point
    ax.text(x, y, data.columns[i], fontsize='14')

**Question:** based on the two previous graphs, can you find a meaning for the two principal components ?

**Answer:**

## 3 Singular Value Decomposition for PCA

We have seen above that PCA can be performed by computing the eigendecomposition of the covariance matrix. An alternative way of performing PCA is with singular value decomposition (SVD) (https://en.wikipedia.org/wiki/Singular_value_decomposition). SVD effectively generalises the eigendecomposition to non-square matrices, factorising a matrix $\mathbf{X} \in \mathbb{R}^{N\times M}$ as $\mathbf{X} = \mathbf{U}\mathbf{S}\mathbf{V}^T$, where $\mathbf{U} \in \mathbb{R}^{N\times N}$ and $\mathbf{V} \in \mathbb{R}^{D\times D}$ are orthonormal, and $\mathbf{S} \in \mathbb{R}^{N\times D}$ is the diagonal matrix of *singular values* with zero rows. Notice that the covariance matrix of $\mathbf{X}$, $$\mathbf{\Sigma} = \mathbf{X}^T\mathbf{X} = \mathbf{V}\mathbf{S}\mathbf{U}^T\mathbf{U}\mathbf{S}\mathbf{V}^T = \mathbf{V}\mathbf{S}^2\mathbf{V}^T,$$ which is an eigenvalue decomposition. Hence, there is a strong link between PCA and SVD: $\mathbf{W} = \mathbf{U}$ are the principal components and $\mathbf{Z} = \mathbf{X}\mathbf{W}^T = \mathbf{U}\mathbf{S}\mathbf{V}^T\mathbf{V} = \mathbf{U}\mathbf{S}$ are the projected data. Therefore, we can use SVD to perform PCA.

### Loading image data

For this task, we will use a classic image from image analysis, "_Lena_". We start by loading and visualising the image with `imread()` from [imageio](https://pypi.org/project/imageio/) and `imshow()` function from `matplotlib`

In [ ]:
from imageio import imread

lena = imread('../data/lena.jpg').astype(float)
plt.imshow(lena, cmap='gray')
plt.axis('off')
plt.show()

### Normalise the data

**Question**: We should first normalise the data by subtracting the mean of each column: $\mathbf{x}'_{:, i} = \mathbf{x}_{:, i} - \frac{1}{N} \sum\limits_{i=1}^{N}x_{ij}$

In [ ]:
# TODO: normalise the data prior to SVD

### Perfom SVD

**Question:** use the `svd()` function from numpy.linalg to perform `svd`.

In [ ]:
# TODO: SVD in numpy. The function should return a tuple U, S, V

**Question:** Reconstruct the *Lena* image from the factorisation. You should plot the reconstruction to check.

In [ ]:
# TODO: reconstruct Lena

### Truncated SVD

To create a lower-rank PCA projection, take the $M$ largest singular values in $\mathbf{S}$, and the corresponding vectors of $\mathbf{U}$ and $\mathbf{V}$. This is equivalent to taking the $M$ largest eigenvectors for PCA, and is known as *truncated SVD*. Note that we are doing PCA on the rows of the image (which are not independent, but this use of PCA as a form of lossy compression is interesting).

In [ ]:
fig = plt.figure(figsize=(6, 6))

for i, M in enumerate([5, 10, 20, 100]):
    # TODO: reconstruct X from truncated matrices
    
    ax = fig.add_subplot(2, 2, i + 1)
    ax.imshow(Z, cmap='gray')
    ax.set_title('No. components = %d' % M)
    ax.axis('off')

plt.tight_layout()
plt.show()

**Question:** What do you observe? How much compression (in terms of floating point numbers stored) is achieved in each case?

**Answer:**